### Memory-Datase
- 채팅메세지를 SQL 데이터베이스 에 저장하는 클래스

**주요기능**
- SQL 데이터베이스를 사용해 채팅 기록을 영구적으로 저장
- 세션 ID 를 통한 대화 구분 및 관리 가능
- SQLAlchemy 를 사용해 다양한 데이터베이스 사용 가능
- 메세지 CRUD 가능

* session_id : 사용자 이름, 이메일 같이 고유 식별자
* connection : 데이터베이스 연결을 지정하는 문자

In [23]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

# SQLChatMessageHistory 객체 생성
chat_message_history = SQLChatMessageHistory(
    session_id="sql_history",
    connection="sqlite:///sqlite.db"  # SQLite 데이터베이스 연결
)


In [24]:
chat_message_history.add_user_message(
    "안녕 안녕 안녕"
)

In [25]:
chat_message_history.add_ai_message(
    "반가워 반가워~"
)

In [28]:
from langchain_core.prompts import ChatPromptTemplate , MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","you are a helpful assistant"),
        
        MessagesPlaceholder(variable_name="chat_history"),
        
        ("human","{question}")
    ]
)

chain = prompt | ChatOpenAI(temperature=0,model='gpt-4o') | StrOutputParser()

In [ ]:
def get_chat_history(user_id,conversation_id):
    return SQLChatMessageHistory(
        table_name=user_id,
        session_id=conversation_id,
        connection="sqlite:///sqlite.db"
    )

In [33]:
from langchain_core.runnables.utils import ConfigurableFieldSpec

config_field = [
    ConfigurableFieldSpec(
        id="user_id",  # 설정값의 고유 식별자
        annotation=str,  # 설정값의 데이터 타입
        name="USER ID",  # 설정값의 이름
        description="Unique identifier for a user",  # 설정값에 대한 설명
        default="",  # 기본값
        is_shared=True  # 여러 대화에서 공유되는 값인지 여부
    ),
    ConfigurableFieldSpec(
        id="conversation_id",  # 설정값의 고유 식별자
        annotation=str,  # 설정값의 데이터 타입
        name="CONVERSATION ID",  # 설정값의 이름
        description="Unique identifier for a conversation",  # 설정값에 대한 설명
        default="",  # 기본값
        is_shared=True  # 여러 대화에서 공유되는 값인지 여부
    )
]


In [35]:
from langchain_core.runnables.history import RunnableWithMessageHistory

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history, # 대화기록을 가져오는 함수
    input_messages_key="question", # 입력 메세지 키
    history_messages_key="chat_history", # 대화 기록 메시지의 키
    history_factory_config=config_field # 대화기록 조회시 참고할 파라미터
)

In [37]:
config = {"configurable":{"user_id":"user","conversation_id":"converastion"}}

chain_with_history.invoke({"question":"안녕로봇 반갑다 안녕 안녕 안녕"},config)

'안녕하세요! 만나서 반갑습니다. 어떻게 도와드릴까요?'

In [40]:
config = {"configurable":{"user_id":"user","conversation_id":"converastion"}}

chain_with_history.invoke({"question":"나의 이름은 안녕로봇이야"},config)

'아, 그렇군요! 안녕로봇님, 만나서 반갑습니다. 어떻게 도와드릴까요?'

In [41]:
config = {"configurable":{"user_id":"user","conversation_id":"converastion2"}}

chain_with_history.invoke({"question":"나의 이름이 뭐야?"},config)

'죄송하지만, 저는 당신의 이름을 알 수 없습니다. 하지만 도움이 필요하시면 언제든지 말씀해 주세요!'